In [ ]:
pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 34.2 MB/s eta 0:

In [ ]:
#@title Mounting Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Importing Data

import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# reading data
train = pd.read_csv('/content/drive/MyDrive/advmla-2023-spring/train.csv')
test = pd.read_csv('/content/drive/MyDrive/advmla-2023-spring/test.csv')

train.head()

<ipython-input-3-71e9aa321c14>:6: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('/content/drive/MyDrive/advmla-2023-spring/train.csv') #.fillna(0)


In [ ]:
#@title Target Count

train['drafted'].value_counts()

0.0    55555
1.0      536
Name: drafted, dtype: int64

In [ ]:
#@title Missing Values

print(train.isna().sum())
print(test.isna().sum())

In [ ]:
#@title Data Cleaning

# function to fix heights
def convert_height(height_str):
    if pd.isna(height_str) or height_str in ['-', 'None', 'So', 'Jr', 'Fr', '0']:
        return None
    elif height_str == 'Apr-00':
        return 4*12
    elif height_str == 'May-00':
        return 5*12
    elif height_str == 'Jun-00':
        return 6*12
    elif height_str == 'Jul-00':
        return 7*12
    elif "'" in height_str:
        feet, inches = height_str.split("'")
        return int(feet) * 12 + int(inches)
    elif '-' in height_str:
        try:
            inch, ft = height_str.split("-")
            if ft == 'Jun':
              return int(inch) + 6*12
            elif ft == 'Jul':
              return int(inch) + 7*12
            elif ft == 'May':
              return int(inch) + 5*12
            elif ft == 'Apr':
              return int(inch) + 4*12
        except ValueError:
            return height_str
    else:
        return height_str

# Apply the function to the 'ht' column
train['ht'] = train['ht'].apply(convert_height)

# Apply the function to the 'ht' column
test['ht'] = test['ht'].apply(convert_height)

79.0    5578
80.0    5498
76.0    5364
77.0    5353
78.0    5126
75.0    5125
74.0    4648
81.0    3988
73.0    3539
72.0    2984
82.0    2491
71.0    1518
70.0    1378
83.0    1119
84.0     653
69.0     598
68.0     242
85.0     201
67.0      95
86.0      88
66.0      40
87.0      40
48.0      20
88.0      11
65.0       8
90.0       7
89.0       4
64.0       4
63.0       3
62.0       3
61.0       2
53.0       1
Name: ht, dtype: int64

In [ ]:
#@title Feature Selection

df_cleaned = train.drop(columns = ['pick', 'Rec_Rank', 'dunks_ratio','type','player_id', 'num'], axis=1)

# df_cleaned = train[['team', 'conf', 'GP', 'dunksmiss_dunksmade', 'usg', 'dunksmade', 'gbpm', 'TO_per', 'dporpag', 'porpag', 'stl_per', 'stops', 'rim_ratio', 'ht', 'drafted']] #'blk_per', 'blk', 'yr',

In [ ]:
df_cleaned['GP_Min'] = df_cleaned['GP']*df_cleaned['Min_per']

In [ ]:
#@title Modelling

from pycaret.classification import *

setup(df_cleaned, target='drafted', categorical_features=['team','conf','year','yr'], fix_imbalance=True, imputation_type='simple', numeric_imputation='mean')

best_model = compare_models()

,Description,Value
0,Session id,3664
1,Target,drafted
2,Target type,Binary
3,Original data shape,"(56091, 58)"
4,Transformed data shape,"(94604, 75)"
5,Transformed train set shape,"(77776, 75)"
6,Transformed test set shape,"(16828, 75)"
7,Numeric features,53
8,Categorical features,4
9,Rows with missing values,21.8%


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9906,0.9826,0.4425,0.5149,0.4715,0.4668,0.4704,73.5850
lightgbm,Light Gradient Boosting Machine,0.9904,0.9722,0.4773,0.4982,0.4860,0.4812,0.4821,17.5040
dummy,Dummy Classifier,0.9904,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.4610
et,Extra Trees Classifier,0.9903,0.9864,0.4802,0.4901,0.4828,0.4780,0.4792,8.9840
rf,Random Forest Classifier,0.9895,0.9808,0.3814,0.4437,0.4088,0.4036,0.4055,71.6290
dt,Decision Tree Classifier,0.9843,0.6698,0.3491,0.2659,0.2999,0.2922,0.2959,9.8220
gbc,Gradient Boosting Classifier,0.9823,0.9788,0.6826,0.3091,0.4251,0.4174,0.4518,132.5890
ada,Ada Boost Classifier,0.9815,0.9704,0.6239,0.2875,0.3926,0.3845,0.4151,26.4930
lr,Logistic Regression,0.9602,0.9891,0.9331,0.1861,0.3100,0.2988,0.4068,16.3890
knn,K Neighbors Classifier,0.9588,0.8279,0.6297,0.1379,0.2262,0.2138,0.2817,6.3030


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
lgb = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9926,0.9751,0.6053,0.6216,0.6133,0.6096,0.6097
1,0.9891,0.9590,0.4211,0.4324,0.4267,0.4211,0.4212
2,0.9888,0.9792,0.3947,0.4167,0.4054,0.3998,0.3999
3,0.9906,0.9808,0.4595,0.5000,0.4789,0.4741,0.4746
4,0.9913,0.9816,0.4595,0.5484,0.5000,0.4957,0.4976
5,0.9896,0.9706,0.4595,0.4474,0.4533,0.4481,0.4481
6,0.9908,0.9669,0.5676,0.5122,0.5385,0.5338,0.5346
7,0.9883,0.9623,0.4324,0.3902,0.4103,0.4044,0.4049
8,0.9931,0.9890,0.6316,0.6486,0.6400,0.6365,0.6366


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
evaluate_model(lgb)

# feature importance - 'team', 'conf', 'GP', 'dunksmiss_dunksmade', 'usg', yr_FR', 'dunksmade', 'gbpm', 'yr_Sr, 'TO_per', 'dporpag', 'yr_Jr, 'porpag', stl_per', 'stops', 'blk_per', 'blk', 'rim_ratio'

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
tunedlgb = tune_model(lgb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9934,0.9894,0.5000,0.7308,0.5937,0.5905,0.6014
1,0.9911,0.9732,0.4211,0.5517,0.4776,0.4732,0.4776
2,0.9916,0.9823,0.3947,0.6000,0.4762,0.4721,0.4827
3,0.9918,0.9858,0.4595,0.5862,0.5152,0.5111,0.5150
4,0.9916,0.9887,0.4324,0.5714,0.4923,0.4882,0.4930
5,0.9890,0.9851,0.4054,0.4167,0.4110,0.4054,0.4055
6,0.9931,0.9790,0.5946,0.6471,0.6197,0.6163,0.6168
7,0.9890,0.9746,0.3784,0.4118,0.3944,0.3888,0.3892
8,0.9934,0.9879,0.5789,0.6875,0.6286,0.6253,0.6276


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
evaluate_model(tunedlgb)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
xgb = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9911,0.9910,0.5000,0.5429,0.5205,0.5161,0.5165
1,0.9898,0.9802,0.5000,0.4750,0.4872,0.4820,0.4822
2,0.9903,0.9759,0.4211,0.5000,0.4571,0.4523,0.4540
3,0.9924,0.9883,0.3784,0.6667,0.4828,0.4792,0.4988
4,0.9913,0.9853,0.4595,0.5484,0.5000,0.4957,0.4976
5,0.9888,0.9841,0.3784,0.4000,0.3889,0.3832,0.3834
6,0.9911,0.9777,0.5405,0.5263,0.5333,0.5288,0.5289
7,0.9893,0.9722,0.4054,0.4286,0.4167,0.4113,0.4114
8,0.9918,0.9908,0.5526,0.5833,0.5676,0.5635,0.5637


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
evaluate_model(xgb)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [16]:
tunedxgb = tune_model(xgb, optimize='AUC')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:13:39
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Extreme Gradient Boosting


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9893,0.9836,0.6316,0.4615,0.5333,0.5281,0.5347
1,0.9855,0.9755,0.5789,0.3492,0.4356,0.4287,0.4429
2,0.9860,0.9851,0.5789,0.3607,0.4444,0.4377,0.4504
3,0.9883,0.9860,0.5405,0.4082,0.4651,0.4593,0.4639
4,0.9896,0.9872,0.5946,0.4583,0.5176,0.5125,0.5169
5,0.9850,0.9834,0.5946,0.3333,0.4272,0.4202,0.4384
6,0.9837,0.9829,0.5946,0.3099,0.4074,0.4000,0.4220
7,0.9865,0.9777,0.5946,0.3667,0.4536,0.4472,0.4606
8,0.9888,0.9853,0.6842,0.4483,0.5417,0.5362,0.5486


In [17]:
evaluate_model(tunedxgb)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
predictions = predict_model(tunedlgb, data=test, raw_score=True)
predictions['drafted'] = predictions['prediction_score_1']
predictions[['player_id', 'drafted']].to_csv('pred_1808.csv', index=False)

[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0


In [ ]:
#@title Chi-square test

from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(train['yr'], train['drafted'])

# Perform the Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-Square value: {chi2}")
print(f"P-value: {p}")

if p < 0.05:
    print("Categorical feature is likely relevant to the target")
else:
    print("Categorical feature is likely not relevant to the target")




Chi-Square value: 38.786348741842694
P-value: 2.1467027304978407e-06
Categorical feature is likely relevant to the target


In [ ]:
#@title Scatterplot Matrix

import seaborn as sns
import matplotlib.pyplot as plt

# Select columns for scatterplot matrix and heatmap
selected_columns = [
       #'GP', 'Min_per', 'Ortg', 'usg', 'eFG', 'TS_per',
       #'ORB_per', 'DRB_per', 'AST_per',
       #'TO_per', 'FTM', 'FTA', 'FT_per',
       #'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per', 'blk_per',
       #'stl_per', 'ftr',

       #'yr', 'ht', 'num', 'porpag', 'adjoe', 'pfr', 'year',
       #'type', 'Rec_Rank', 'ast_tov', 'rimmade', 'rimmade_rimmiss',
       #'midmade','midmade_midmiss', 'rim_ratio', 'mid_ratio', 'dunksmade',
       #'dunksmiss_dunksmade', 'dunks_ratio', 'pick', 'drtg', 'adrtg',
       #'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm', 'mp', 'ogbpm',
       'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk', 'pts',
        'drafted']

# Create a scatterplot matrix (pairplot)
#sns.pairplot(train[selected_columns])
#plt.show()

#best - twoPM 0.22, TPM 0.12, PORPAG 0.26, adjoe 0.11, rec_rank 0.19, rimmade 0.21, midmade 0.2, dunksmade 0.24, adrtg 0.11,

# Create a heatmap
corr_matrix = train[selected_columns].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()